# 14 - Testing, Debugging, and Exceptions

## Testing Output Sent to stdout
Using the unittest.mock module’s patch() function, it’s pretty simple to mock out sys.stdout for just a single test, and put it back again.

In [1]:
def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)


In [2]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)
        with patch('sys.stdout', new=StringIO()) as fake_out:
            urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)


In [3]:
import unittest

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## Patching Objects in Unit Tests

In [4]:
from unittest import TestCase
from unittest.mock import patch

def func():
    pass

class TestFunc(TestCase):
    @patch('__main__.func')
    def test_func(self, mock_func):
        x = 3
        func(x)  # uses patched example.func
        mock_func.assert_called_with(x)


In [5]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


If necessary, you can stack decorators and context managers to patch multiple objects.

In [6]:
from unittest import TestCase
from unittest.mock import patch


def func1():
    pass

def func2():
    pass

def func3():
    pass


class TestFuncPatchy(TestCase):
    @patch('__main__.func1')
    @patch('__main__.func2')
    @patch('__main__.func3')
    def test_func_patchy(self, mock1, mock2, mock3):
        x = 3
        func1(x)
        func2(x)
        func3(x)
        mock1.assert_called_with(x)
        mock2.assert_called_with(x)
        mock3.assert_called_with(x)


In [7]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


## Testing for Exceptional Conditions in Unit Tests
You want to write a unit test that cleanly tests if an exception is raised.

In [8]:
import unittest

def parse_int(s):
    return int(s)

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')


In [9]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


Suppose that we need to test an exception's value.

In [11]:
import errno

class TestIO(unittest.TestCase):
    def test_file_not_found(self):
        try:
            f = open('/file/not/found')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)
        else:
            self.fail('IOError not raised')


In [12]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


## Logging Test Output to a File

In [13]:
import sys

def main(out=sys.stderr, verbosity=2):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromModule(sys.modules[__name__])
    unittest.TextTestRunner(out,verbosity=verbosity).run(suite)

if __name__ == '__main__':
    with open('testing.out', 'w') as f:
        main(f)


In [14]:
%%bash
cat testing.out

test_bad_int (__main__.TestConversion) ... ok
test_func (__main__.TestFunc) ... ok
test_func_patchy (__main__.TestFuncPatchy) ... ok
test_file_not_found (__main__.TestIO) ... ok
test_url_gets_to_stdout (__main__.TestURLPrint) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK


## Skipping or Anticipating Test Failures

In [15]:
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)
 
    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')
 
    @unittest.skipIf(os.name=='posix', 'Not supported on Unix')
    def test_2(self):
        import winreg

    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)
 
    @unittest.expectedFailure
    def test_4(self):
        self.assertEqual(2+2, 5)


In [16]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

......s.sx
----------------------------------------------------------------------
Ran 10 tests in 0.009s

OK (skipped=2, expected failures=1)


## Handling Multiple Exceptions
If you can handle different exceptions all using a single block of code (tuple).